# Analyse data

start date:
https://www.data.jma.go.jp/sakura/data/sakura003_06.html
    
full bloom:
https://www.data.jma.go.jp/sakura/data/sakura004_06.html
    
temp data:
March temp data?    
    
http://www.st.nanzan-u.ac.jp/info/gr-thesis/ms/2007/03mm081.pdf

https://www.economist.com/graphic-detail/2017/04/07/japans-cherry-blossoms-are-emerging-increasingly-early

https://www.washingtonpost.com/graphics/2018/national/early-spring/?utm_term=.5ce6e78f16a4

In [25]:
import pandas as pd
import re
import numpy as np
from datetime import datetime


import math
import statistics

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
matplotlib.rcParams['pdf.fonttype'] = 42

%matplotlib inline

pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

from datetime import datetime, timedelta

In [2]:
df = pd.read_csv('data/sakura_complete_just_dates.csv')

## SOMA HELP

In [3]:
cols = list(df.columns[df.columns.str.contains("_start") | df.columns.str.contains("_full")])
cols.append('city_eng')

# Let's melt those into a smaller long dataframe
melted = df[cols].melt(id_vars='city_eng', value_name='doy', var_name='year')
melted['kind'] = melted.year.str.split("_").str.get(1)
melted['year'] = melted.year.str.split("_").str.get(0)

melted.head()

,city_eng,year,doy,kind
0,Wakkanai,2011,519.0,start
1,Asahikawa,2011,509.0,start
2,Abashiri,2011,516.0,start
3,Sapporo,2011,507.0,start
4,Obihiro,2011,503.0,start


In [17]:
melted_median = melted.drop(columns='year').groupby(['city_eng', 'kind']).median().reset_index()
melted_median.head(4)

# or: 
# melted.groupby(['city_eng', 'kind']).doy.median().reset_index() 

,city_eng,kind,doy
0,Abashiri,full,511.0
1,Abashiri,start,508.0
2,Akita,full,420.0
3,Akita,start,416.0


In [42]:
pivoted = pd.pivot(melted_median, index='city_eng', columns='kind', values='doy').reset_index()

In [43]:
pivoted.head(1)

kind,city_eng,full,start
0,Abashiri,511.0,508.0


In [44]:
pivoted['full'] = pivoted.full.apply(lambda doy: datetime(2000, 1, 1) + timedelta(days=doy))
pivoted['start'] = pivoted.start.apply(lambda doy: datetime(2000, 1, 1) + timedelta(days=doy))

In [45]:
pivoted

kind,city_eng,full,start
0,Abashiri,2001-05-26 00:00:00,2001-05-23
1,Akita,2001-02-24 00:00:00,2001-02-20
2,Aomori,2001-03-01 00:00:00,2001-02-24
3,Asahikawa,2001-05-19 00:00:00,2001-05-18
4,Choshi,2001-02-09 00:00:00,2000-11-26
5,Fukui,2001-02-10 00:00:00,2000-11-27
6,Fukuoka,2000-11-26 00:00:00,2000-11-17
7,Fukushima,2001-02-14 00:00:00,2001-02-10
8,Gifu,2001-02-05 00:00:00,2000-11-19
9,Hakodate,2001-05-16 00:00:00,2001-03-04
